In [2]:
import pandas as pd

In [215]:
df = pd.read_csv('Moped Date_normal values_with30subtopic.csv', index_col = 0)
df.head()

/Users/reikanakatsu/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (3,4,5,6,7,8,9,11,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,RENTAL_ID,RATING,CATEGORY,SYSTEM_IDENTIFIER,SYSTEM_STARTED_AT,RENTER_EXTERNAL_ID,FLEET_RESOURCE_ID,DISCOUNT_RATE_NAME,RENTAL_PASS_NAME,RENTER_RIDE_NUM,...,tokens,words,description3,2gram,3gram,Group,Topic,SubTopic,SubTopicNumber,Number_of_topics
0,6708670.0,1.0,trouble_starting_ride,newyork,2021-07-08 21:34:23,5e752dbb7a4b80000fa66f65,5dfa2dc6446423000ed6666f,Default,NaN,21.0,...,"['the', 'way', 'it', 'was', 'parked', 'one', '...","['way', 'parked', 'one', 'tire', 'lifted', 'st...",way parked one tire lifted stuck think wrong k...,"[('way', 'parked'), ('parked', 'one'), ('one',...","[('way', 'parked', 'one'), ('parked', 'one', '...",['Equipment/Engine'],['Tire'],['Flat tire'],['17'],1
1,157827.0,NaN,NaN,newyork,2019-06-30 17:35:53,5e752dc17a4b80000fa684c9,5d9ccaf2647f4d001c96a12d,Default,NaN,2.0,...,"['i', 'can', 't', 'end', 'the', 'ride', 'i', '...","['end', 'ride', 'end', 'ride', 'need', 'finish...",end ride end ride need finish trip not working...,"[('end', 'ride'), ('ride', 'end'), ('end', 'ri...","[('end', 'ride', 'end'), ('ride', 'end', 'ride...",['Ride'],['App'],['Cannot end ride'],['27'],1
2,4364285.0,NaN,NaN,newyork,2020-09-12 21:43:26,5eaf73c7c1c50b001189b503,5ed02cdd52bfca01f7a4748a,Default,NaN,9.0,...,"['the', 'ride', 'was', 'minutes', 'but', 'i', ...","['ride', 'minute', 'restart', 'phone', 'app', ...",ride minute restart phone app connect end ride...,"[('ride', 'minute'), ('minute', 'restart'), ('...","[('ride', 'minute', 'restart'), ('minute', 're...",['Ride'],['App'],['Cannot end ride'],['27'],1
3,1547922.0,NaN,NaN,newyork,2020-03-02 13:38:53,5e75330d7a4b800010a6a972,5d9ccafa647f4d001c96a28d,Default,NaN,15.0,...,"['this', 'bike', 'won', 't', 'start', 'the', '...","['bike', 'start', 'engine', 'ride', 'able', 's...",bike start engine ride able start wan na pay,"[('bike', 'start'), ('start', 'engine'), ('eng...","[('bike', 'start', 'engine'), ('start', 'engin...",['Ride'],['Moped'],['Moped Not Start'],['29'],1
4,3250027.0,NaN,NaN,newyork,2020-06-26 10:19:33,5eb8211fa655dd0011c320b0,5ed02cdd52bfca01f7a476c2,Default,NaN,20.0,...,"['i', 'did', 'not', 'ride', 'the', 'bike', 'th...","['not', 'ride', 'bike', 'far', 'charge', 'not'...",not ride bike far charge not correct ride ende...,"[('not', 'ride'), ('ride', 'bike'), ('bike', '...","[('not', 'ride', 'bike'), ('ride', 'bike', 'fa...",['Ride'],['Moped'],"['Moped Not Start', 'Moped Damaged/Malfunction']","['29', '28']",2


In [240]:
#convert string list to an actual list
df['SubTopic_list'] = df['SubTopic'].apply(lambda x: x.strip('][').replace("'", '').split(', '))

In [157]:
topics= df['SubTopic_list'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
rating= df['RATING'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
topics.head(10)

,value
index,
0,Flat tire
1,Cannot end ride
2,Cannot end ride
3,Moped Not Start
4,Moped Not Start
5,Moped Not Start
6,Moped Not Start
7,Cannot end ride
8,Brake broken


In [165]:
merge_rate=pd.merge(
    pd.merge(
        topics,
        rating,
        left_index=True,
        right_index=True),
    df[['RENTAL_ID']],
    left_index=True,
    right_index=True).rename(columns={'value_x': 'Topics', 'value_y': 'Rating'})
merge_rate

,Topics,Rating,RENTAL_ID
0,Flat tire,1.0,6708670.0
9,Cannot end ride,1.0,9406530.0
11,Stop in the middle,1.0,7454776.0
17,Brake broken,1.0,5613367.0
21,Flat tire,1.0,6303182.0
...,...,...,...
189482,Moped Not Start,1.0,8169124.0
189487,App not recognize parking zone,1.0,8538431.0
189491,Only small helmets,4.0,5949418.0
189492,No or broken mirror,3.0,9147940.0


In [169]:
df_rating_group = merge_rate.groupby("Topics")["Rating"].mean().sort_values(ascending=True)
df_rating_group

Topics
Spanish                                  1.000000
Flat tire                                1.550712
Stop in the middle                       1.634790
Moped Not Start                          1.644231
Throttle not work                        1.952565
shaking                                  1.955556
Handle unstable                          1.955556
Moped Damaged/Malfunction                1.982179
Kickstand broken                         2.034483
Cannot end ride                          2.035088
App not recognize parking zone           2.066131
Not clear                                2.094162
No or low battery                        2.121011
Overcharged (Complaint)                  2.144162
Turning light not working                2.492579
Pricey (Complaint)                       2.494670
Scratched/graffiti                       2.507163
Brake broken                             2.517382
Wheel broken/wobble                      2.519164
Pass not applied                         2.

## method two

In [232]:
df_all_rate=df[['RATING','SubTopic']].dropna()

In [235]:
df2 = df_all_rate.SubTopic.str.split(',').apply(pd.Series)
df2.index = df_all_rate.set_index(['RATING', 'SubTopic']).index
df2=df2.stack().reset_index(['RATING', 'SubTopic'])

In [236]:
df2

,RATING,SubTopic,0
0,1.0,['Flat tire'],['Flat tire']
0,1.0,['Cannot end ride'],['Cannot end ride']
0,1.0,['Stop in the middle'],['Stop in the middle']
0,1.0,['Brake broken'],['Brake broken']
0,1.0,"['Flat tire', 'Moped Damaged/Malfunction']",['Flat tire'
...,...,...,...
0,1.0,['Moped Not Start'],['Moped Not Start']
0,1.0,['App not recognize parking zone'],['App not recognize parking zone']
0,4.0,['Only small helmets'],['Only small helmets']
0,3.0,['No or broken mirror'],['No or broken mirror']


In [237]:
df2[0] = tuple(df2[0].apply(lambda x: x.strip('][').replace("'", '').split(', ')))

In [238]:
df2

,RATING,SubTopic,0
0,1.0,['Flat tire'],"(Flat tire,)"
0,1.0,['Cannot end ride'],"(Cannot end ride,)"
0,1.0,['Stop in the middle'],"(Stop in the middle,)"
0,1.0,['Brake broken'],"(Brake broken,)"
0,1.0,"['Flat tire', 'Moped Damaged/Malfunction']","(Flat tire,)"
...,...,...,...
0,1.0,['Moped Not Start'],"(Moped Not Start,)"
0,1.0,['App not recognize parking zone'],"(App not recognize parking zone,)"
0,4.0,['Only small helmets'],"(Only small helmets,)"
0,3.0,['No or broken mirror'],"(No or broken mirror,)"


In [239]:
df_rating_group = df2.groupby(0)["RATING"].mean().sort_values(ascending=True)
df_rating_group

0
( Not clear,)                                1.000000
( No or low battery,)                        1.000000
(Spanish,)                                   1.000000
( Moped Not Start,)                          1.432203
( Throttle not work,)                        1.445230
( Cannot end ride,)                          1.481481
( Flat tire,)                                1.500000
(Flat tire,)                                 1.551121
( Overcharged (Complaint),)                  1.627907
(Stop in the middle,)                        1.634790
(Moped Not Start,)                           1.656043
( App not recognize parking zone,)           1.666667
( Moped Damaged/Malfunction,)                1.749288
( Handle unstable,)                          1.764706
( shaking,)                                  1.955556
(Handle unstable,)                           1.958264
( Wheel broken/wobble,)                      2.000000
(Throttle not work,)                         2.008584
(Kickstand broken,)       